<a href="https://colab.research.google.com/github/sallywang147/SCInvarinfer/blob/main/%5BNQ_stanford_format%5D_llama_for_smart_contracts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install --quiet accelerate
!pip install -i --quiet https://test.pypi.org/simple/ bitsandbytes
!pip install --quiet sentencepiece
!pip install --quiet git+https://github.com/huggingface/transformers
!pip install --quiet rich[jupyter]
!pip install --quiet deepspeed
!pip install --quiet openai
!pip install --quiet gradio
!pip install --quiet peft
!pip install --quiet --upgrade datasets
!pip install --quiet simplejson
!pip install --quiet accelerate
!pip install --quiet bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 5.8 MB/s eta 0:00:00
Looking in indexes: --quiet, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 62.4 MB/s eta 0:00:00
  ERROR: Cannot unpack file /tmp/pip-unpack-8nf9c1pn/simple.html (downloaded from /tmp/pip-req-build-zdsevgr4, content-type: text/html); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-zdsevgr4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.4/765.4 KB 12.8 MB/s

##Vanilla Llama v. 8bit_Quantized Llama##

vanilla llama is slightly more capable than quantized llama, but not by much. For example, vanilla llama tends to repeat the same information less. 

In [4]:
import os
import numpy as np
import accelerate
import bitsandbytes
import transformers
import pandas as pd
import plotly.express as px
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import AutoTokenizer, LlamaTokenizer, LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained(
            "sallywww/trained_llama_stanford_format" )

tokenizer = LlamaTokenizer.from_pretrained(
            "sallywww/trained_llama_stanford_format"
        )
#model = LlamaForCausalLM.from_pretrained('sallywww/Llama-7B')
#tokenizer = AutoTokenizer.from_pretrained('sallywww/Llama-7B')

#model.push_to_hub('sallywww/Llama-7B')
#tokenizer.push_to_hub('sallywww/Llama-7B')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:

prompt = "a loop invariant of smart contracts is " 
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=100)
tokenizer.batch_decode(generate_ids, 
                       skip_special_tokens=True, 
                       temperature=0.8, 
                       top_p=0.95, 
                       top_k=50, 
                       repeat_penalty=0.2,
                       clean_up_tokenization_spaces=False)[0]


' a loop invariant of smart contracts is  assert(x == y);'

##Inference with trained llama##

Let's make some inferences without targeted instruction finetuning yet

In [53]:
contract = 'suicide.sol'
lora_param = '/content/drive/MyDrive/lora-llama7b_insft_100e'
test_file = open(f'/content/drive/MyDrive/experiments/baseline benchmark/{contract}', "r")
test_data = test_file.read()

In [54]:
import argparse
import random
import torch
#import gradio as gr
import torch.nn as nn
#import bitsandbytes
from transformers import LlamaForCausalLM, LlamaTokenizer

def infer_invariants(model, tokenizer, test_data, \
                     temperature, top_p, top_k,  max_new_tokens):
  model.cuda()
  model.eval()
  inputs = tokenizer(test_data, return_tensors="pt")
  with torch.no_grad():
      outputs = model.generate(input_ids=inputs["input_ids"].cuda(), \
       temperature=temperature, 
       top_p=top_p, 
       top_k=top_k, 
      max_new_tokens=max_new_tokens)
      invariants = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
  return invariants

In [55]:
output = infer_invariants(model=model, 
                          tokenizer=tokenizer, 
                          test_data=test_data,
                          temperature=None, 
                          top_p=None, 
                          top_k=None, 
                          max_new_tokens=670)
print(output.strip())
with open(f"/content/drive/MyDrive/experiments/evaluation/noPEFT_finetuned_llama/{contract}", "w", encoding="utf-8") as file:  
    file.write(output.strip()+"\n")
file.close()

1  // SPDX-License-Identifier: MIT
2  pragma solidity >=0.4.24 <0.6.0;
3  
4  contract suicide{
5         function initOwners(address[] _owners,
6                             uint _required){
7         if (m_numOwners > 0) throw;
8         m_numOwners = _owners.length + 1;
9         m_owners[1] = uint(msg.sender);
10         m_ownerIndex[uint(msg.sender)] = 1;
11         m_required = _required;
12 
13         }
14 
15         function suicide(address _to) {
16             uint ownerIndex = m_ownerIndex[uint(msg.sender)];
17             if (ownerIndex == 0) return;
18             var pending = m_pending[sha3(msg.data)];
19             if (pending.yetNeeded == 0) {
20             pending.yetNeeded = m_required;
21             pending.ownersDone = 0;
22             }
23             uint ownerIndexBit = 2**ownerIndex;
24             if (pending.ownersDone   ownerIndexBit == 0) {
25             if (pending.yetNeeded <= 1)
26                 suicide(_to);
27             else {
28            

In [ ]:
for d in os.listdir(): 
  if os.path.isdir(d) and d.startswith('lora-'):
      print(d)
      lora_model = d
  
inference = generate_text(
                lora_param,
                test_data,
                #temperature=0.8,
                #top_p=0.95,
                #top_k=50,
                #repeat_penalty=0.02,
                max_new_tokens=750)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Instruction Finetuning##

We're mostly going to change the dataset format here and add additional instructions for llama 

##GPT-style finetuning: OOM error persisted##

In [ ]:
from google.colab import auth
from google.auth import default
from numpy import random
import pandas as pd
import gspread
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
#defining my worksheet
worksheet = gc.open('short_invariants_for_gpt').sheet1
#get_all_values gives a list of rows
rows = worksheet.get_all_values()
#Convert to a DataFrame 
cols = ['Target']
df = pd.DataFrame(rows, columns=cols)
df

In [ ]:
model_params={
    "MODEL":"llama_model",             # model_type: t5-large
    "MAX_LENGTH": 3036,  # max length of source text
   # "SEED": random.randint(1000)    # randomized seeds to shuffle test set

}

In [ ]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from rich.console import Console
from tqdm import tqdm, trange
import torch.nn.functional as F
import random
import torch
import csv

class GPTDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, target_label, truncate=False, \
               gpt2_type=model, \
               max_length=2048):
    self.tokenizer = tokenizer
    self.target_tokens = []

    for row in df['Target']:
        self.target_tokens.append(torch.tensor(
            self.tokenizer.encode(f"<|{target_label}|>{row[:max_length]}<|endoftext|>")
            ))  
    if truncate:
            self.target_tokens = self.target_tokens[:20000]
    self.length = len(self.target_tokens)   

  def __len__(self):
    return self.length

  def __getitem__(self, index):
    return self.target_tokens[index]   

In [ ]:
#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [ ]:
def train(
    dataset, model, tokenizer,
    batch_size=0.1, epochs=3, lr=2e-5,
    max_seq_len=2048, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False, save_model_on_epoch=False,
):
    acc_steps = 10
    device=torch.device("cuda")
    model = model
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None
    loss_list = []
    epoch_list = []
    for epoch in range(epochs):
        total_loss = []
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to('cuda')
            outputs = model(input_tensor, labels=input_tensor)  
            loss = outputs[0] 
            total_loss.append(float(loss.item()))       
            loss.backward()
            torch.cuda.empty_cache()                      
            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()
  
            accumulating_batch_count += 1
            input_tensor = None

        #training_logger.add_row(str(epoch), str(np.mean(total_loss)))       
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
        loss_list.append(np.mean(total_loss))
        epoch_list.append(epoch) 
        print(f"for epoch {epoch} the loss is {np.mean(total_loss)}\n")
   # console.print(training_logger)   
   # plot_loss(epoch_list, loss_list)
    return model

In [ ]:
def fine_tune_GPT2(df, model_params):   
  #console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  dataset = GPTDataSetClass(df['Target'], truncate=False, gpt2_type=model) 
  #Get the tokenizer and model
  #tokenizer = GPT2Tokenizer.from_pretrained(model_params["MODEL"])
 # model = GPT2LMHeadModel.from_pretrained(model_params["MODEL"])  
  trained_model = train(dataset, model, tokenizer)
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join('/content/output', "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)
  console.print(f"""[Model] Model saved @ {os.path.join('/content/output', "model_files")}\n""")
  
  # logging
  console.log(f"[Data]: Reading Raw data...\n")


  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 

  console.print(f"FULL Dataset: {df.shape}")
  return trained_model, tokenizer


In [ ]:

def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=2048, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text + '\n')
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text + '\n')
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(model, tokenizer, test_data):
  generated_code = []
  for i in range(len(test_data)):    
    x = generate(model.to('cpu'), tokenizer, test_data['Target'][i], entry_count=1)
    generated_code.append(x)
  return generated_code

#Run the functions to generate the lyrics

def test_fine_tuned_gpt2(model, tokenizer, df): 
    test_set = df.sample(n=1)
    df = df.loc[~df.index.isin(test_set.index)]

    #Reset the indexes
    test_set = test_set.reset_index()
    df = df.reset_index()

    #For the test set only, keep last 20 words in a new column, then remove them from original column
    test_set['Target'] = test_set['Target'].str.split().apply(' '.join)
    generated_code = text_generation(model, tokenizer, test_set)
    print(generated_code)


In [ ]:
#To solve CUDA out of memory error
import gc
gc.collect()
torch.cuda.empty_cache()
os.environ['CUDA_VISIBLE_DEVICES']='0, 1, 2, 3, 4, 5, 6, 7, 8'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
trained_model, trained_tokenizer = fine_tune_GPT2(df, model_params)

In [ ]:
import json
import os

import deepspeed
import torch
from beartype import beartype
from beartype.typing import Tuple
from einops import rearrange
from torch.utils.data import DataLoader, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)



  
local_rank, world_size = setup_model_parallel()
            # use load_model_test for testing
  self.model, self.tokenizer = load_model(
   ckpt_dir=model,
   tokenizer_path=tokenizer,
   local_rank=local_rank,
   world_size=world_size,
   froze_embeddings=config.froze_embeddings,
   use_fairscale=config.use_fairscale,
   max_batch_size=config.batch_size,
    )
        elif config.model in hf_models_seq_2_seq:
    
            # galactica tokenizer eos_token is None
            if self.tokenizer.eos_token is None:
                self.tokenizer.eos_token = "</s>"
                self.tokenizer.eos_token_id = 0
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id
            self.model = AutoModelForCausalLM.from_pretrained(
                config.model,
            )
            self.model.to(config.device)

        # load the model from model_folder
        self.load()

    @beartype
    def load(self) -> None:
        """Load the model from the path

        Args:
            path (str): path to the model
        """
        path = ModelLoader.check_model_path(
            config=self.config,
            is_checkpoint=False,
            current_epoch=None,
        )
        if path is not None:
            print("Loading ...")
            model_dict = torch.load(path)
            self.model.load_state_dict(model_dict["model"])
            self.head.load_state_dict(model_dict["head"])

    @beartype
    def save(self) -> None:
        """Save the model to the path

        Args:
            path (Optional[str], optional): Path to store the model.
                Defaults to None.
        """
        model_folder, model_name, path = ModelLoader.get_model_path(
            config=self.config,
            is_checkpoint=False,
            current_epoch=None,
        )
        print(f"Saving model to {path} ...")
        torch.save(
            {"model": self.model.state_dict(), "head": self.head.state_dict()},
            path,
        )

    def parameters(self, **kwargs):
        """Return the parameters of the model

        Args:
            **kwargs:
        """
        return self.model.parameters()

    @beartype
    def forward(
        self, sequences: torch.Tensor, sequences_mask: torch.Tensor
    ) -> torch.Tensor:
        """Generate logits to have probability distribution over the vocabulary
            of the actions

        Args:
            sequences (torch.Tensor): Sequences of states and actions used to
                    compute token logits for the whole list of sequences
            attention_mask (torch.Tensor): Mask for the sequences attention

        Returns:
            logits (torch.Tensor): Logits for the actions taken
        """
        model_output = self.model.forward(
            sequences, attention_mask=sequences_mask
        )
        # need to return logits for the actions
        if self.config.model in hf_models_causal_lm:
            model_output = model_output.logits
        if self.config.debug:
            print("ActorModel.forward")
            print("model_output_logits shape", model_output.shape)
            print("model_output logits", model_output)
        return model_output

    @beartype
    @torch.no_grad()
    def generate(
        self, states: torch.Tensor, state_mask: torch.Tensor
    ) -> Tuple:
        """Generate actions and sequences=[states, actions] from state
            (i.e. input of the prompt generator model)

        Args:
            state (torch.Tensor): the input of the user
            state_mask (torch.Tensor): Mask for the state input (for padding)

        Returns:
            actions (torch.Tensor): Actions generated from the state
            sequences (torch.Tensor): Sequences generated from the
                state as [states, actions]
        """
        temperature = self.config.temperature
        # max sequence length for the actor (i.e. prompt + completion)
        # from config file - it depends by the model used
        max_sequence_length = self.config.max_sequence_length
        # max tokens generated by the actor (completion only) from config file
        max_tokens = self.config.max_tokens
        # temperature for the actor
        max_generation_possible = max_sequence_length - states.shape[1]
        # take the minimum between the maximum token that you want to generate
        # and the token that is possible to generate given the maximum sequence
        # supported
        max_completion = min(max_tokens, max_generation_possible)
        if max_completion <= 0:
            raise ValueError(
                "The maximum completion available is <= 0 the prompt is too "
                + "long w.r.t the model sequence length"
            )
        # the max_length is then the input length + the completion length
        max_length = states.shape[1] + max_completion
        # generate
        sequences = self.model.generate(
            input_ids=states,
            attention_mask=state_mask,
            temperature=temperature,
            max_length=max_length,
        )
        actions = sequences[:, states.shape[1] :]  # noqa E203
        if self.config.debug:
            print("ActorModel.generate")
            print("state", states)
            print("state shape", states.shape)
            print("sequence shape", sequences.shape)
            print("sequence", sequences)
            print("actions shape", actions.shape)
            print("actions", actions)
        return actions, sequences


class ActorDataset(Dataset):
    """Dataset for the pretraining of the actor model
    read a json file with the following format:
    [
        {
            "user_input": "..."
            "completion": "..."
        },
        ...
    ]
    Where:
        user_input: the input of the user
        completion: the output of the user
    """

    def __init__(self, path: str) -> None:
        self.path = path
        with open(path, "r") as f:
            data = json.load(f)
            self.data = [d["user_input"] + " " + d["completion"] for d in data]
        self.len = len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(
        self,
    ):
        return self.len


class ActorTrainer:
    """Used to pre-train the actor model to generate better prompts.

    Args:
        config (ConfigActor): Configuration for the actor model

    Attributes:
        config (ConfigActor): Configuration for the actor model
        model (ActorModel): Actor model
        loss_function (torch.nn.CrossEntropyLoss): Loss function
        optimizer (torch.optim.Adam): Optimizer
        validation_flag (bool): Flag to indicate if the validation dataset
            is provided
        training_stats (TrainingStats): Training statistics

    Methods:
        train: Train the actor model
    """

    def __init__(self, config: ConfigActor) -> None:
        # load the model, optimizer, loss function and config
        self.config = config
        self.model = ActorModel(config)
        self.loss_function = torch.nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(
            self.model.parameters(), lr=config.lr
        )

        # check checkpoint, datasets and other data
        self.validation_flag = False
        self.training_stats = TrainingStats()
        if config.validation_dataset_path is not None:
            self.validation_flag = True

        # create dataloaders
        self.train_dataset = ActorDataset(config.train_dataset_path)
        self.train_dataloader = DataLoader(
            self.train_dataset, batch_size=config.batch_size
        )
        if self.validation_flag:
            self.eval_dataset = ActorDataset(config.validation_dataset_path)
            self.validation_dataloader = DataLoader(
                self.eval_dataset, batch_size=config.batch_size
            )

        # initialize deepspeed
        self.model_engine = None
        if config.deepspeed_enable is True:
            if config.deepspeed_config_path is None:
                raise ValueError(
                    "DeepSpeed config path is None, but deepspeed is enabled"
                )
            if os.path.exists(config.deepspeed_config_path) is False:
                raise ValueError(
                    f"DeepSpeed config path {config.deepspeed_config_path}"
                    f"does not exist"
                )
            (
                self.model_engine,
                self.optimizer,
                self.train_dataloader,
                _,
            ) = deepspeed.initialize(
                args=None,
                model=self.model,
                model_parameters=self.model.parameters(),
                training_data=self.train_dataloader,
                config=self.config.deepspeed_config_path,
            )

    @beartype
    def save_checkpoint(
        self,
        current_epoch: int,
        current_step: int,
        max_epochs: int,
        max_steps: int,
    ) -> None:

        print(
            f"Saving checkpoint for epoch {current_epoch + 1}, "
            f"step {current_step + 1} ..."
        )
        model_folder, model_name, path = ModelLoader.get_model_path(
            config=self.config,
            is_checkpoint=True,
            current_epoch=current_epoch,
            current_step=current_step,
            max_epochs=max_epochs,
            max_steps=max_steps,
        )
        if os.path.exists(path):
            os.remove(path)
        torch.save(
            {
                "state_dict": self.model.state_dict(),
                "optim_state_dict": self.optimizer.state_dict(),
                "training_stats": self.training_stats,
                "epoch": current_epoch,
                "step": current_step,
            },
            path,
        )

    @beartype
    def load_checkpoint(
        self,
    ) -> Tuple[int, int]:
        """Load a checkpoint from the model folder"""

        print("Looking for checkpoints...")
        path = ModelLoader.check_model_path(
            config=self.config,
            is_checkpoint=True,
            current_epoch=None,
        )
        if path is not None:
            print("Loading ...")
            checkpoint = torch.load(path)
            epoch = checkpoint["epoch"]
            self.model.load_state_dict(checkpoint["state_dict"])
            self.optimizer.load_state_dict(checkpoint["optim_state_dict"])
            self.trainign_stats = checkpoint["training_stats"]
            step = checkpoint["step"]
            return epoch, step + 1  # return the next episode to train
        return 0, 0

    def train(
        self,
    ) -> None:
        print("Start Actor Model Pretraining")

        # get config parameters
        batch_size = self.config.batch_size
        epochs = self.config.epochs
        device = self.config.device
        checkpoint_steps = self.config.checkpoint_steps

        # compute the number of iterations
        n_iter = int(len(self.train_dataset) / batch_size)

        # load model_checkpoint
        start_epoch, start_step = self.load_checkpoint()

        # counter for the checkpoint
        cnt_checkpoint = 1

        # traing loop
        for epoch in range(start_epoch, epochs):
            self.model.train()
            for i, input_output in enumerate(self.train_dataloader):
                if i < start_step:
                    continue
                with torch.no_grad():
                    input_output_tokenized = self.model.tokenizer(
                        input_output,
                        return_tensors="pt",
                        padding=True,
                    )
                    training_output = input_output_tokenized["input_ids"][
                        :, 1:
                    ]
                    training_input = input_output_tokenized["input_ids"][
                        :, :-1
                    ]
                    attention_mask = input_output_tokenized["attention_mask"][
                        :, :-1
                    ]
                    training_output = training_output.to(device)
                    training_input = training_input.to(device)
                    attention_mask = attention_mask.to(device)

                # forward pass
                if self.config.deepspeed_enable:
                    est_output = self.model_engine(
                        training_input, attention_mask
                    )
                else:
                    est_output = self.model(training_input, attention_mask)
                est_output = rearrange(est_output, "b s v -> (b s) v")
                training_output = rearrange(training_output, "b s -> (b s)")
                loss = self.loss_function(est_output, training_output)
                self.training_stats.training_loss.append(loss.item())

                # backward pass
                if self.config.deepspeed_enable:
                    self.model_engine.backward(loss)
                    self.model_engine.step()
                else:
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()

                # print progress
                if i % self.config.iteration_per_print == 0:
                    print(
                        f"Epoch: {epoch+1}/{epochs}, "
                        f"Iteration: {i+1}/{n_iter}, "
                        f"Training Loss: {loss}"
                    )
                # save checkpoint periodically
                if cnt_checkpoint % checkpoint_steps == 0:
                    self.save_checkpoint(epoch, i, epochs, n_iter)
                    cnt_checkpoint = 1
                else:
                    cnt_checkpoint += 1

            if self.validation_flag:
                self.model.eval()
                for i, input_output in enumerate(self.validation_dataloader):
                    input_output_tokenized = self.model.tokenizer(
                        input_output, return_tensors="pt", padding=True
                    )
                    validation_output = input_output_tokenized["input_ids"][
                        :, 1:
                    ]
                    validation_input = input_output_tokenized["input_ids"][
                        :, :-1
                    ]
                    attention_mask = input_output_tokenized["attention_mask"][
                        :, :-1
                    ]

                    # forward pass
                    est_output = self.model.forward(
                        validation_input, attention_mask
                    )
                    validation_output = rearrange(
                        validation_output, "b s -> (b s)"
                    )
                    est_output = rearrange(est_output, "b s v -> (b s) v")
                    loss = self.loss_function(est_output, validation_output)
                    self.training_stats.validation_loss.append(loss.item())
                    # print progress
                    if i % self.config.iteration_per_print == 0:
                        print(
                            f"Epoch: {epoch+1}/{epochs}, "
                            f"Iteration: {i+1}/{n_iter}, "
                            f"Validation Loss: {loss}"
                        )
        self.model.save()
        print("Training Finished ")

In [ ]:
def generate_test_contrct(path, ratio): 
  col = ['Target']
  f = open(path, "r")
  file = f.read()
  test_df = pd.DataFrame([file], columns=col)
  program_length = len(test_df['Target'][0].split())
  prompt_ratio = ratio
  prompt_length = int(prompt_ratio * program_length)
  return test_df, prompt_length

def truncate_test(df, prompt_length):
  copy_1 = df.copy(deep=True)
  copy_2 = df.copy(deep=True)
  #true
  a = copy_1['Target'].str.split().str[-prompt_length:].apply(' '.join)[0]
  #masked out program 
  b = copy_2['Target'].str.split().str[:-prompt_length].apply(' '.join)[0]
  return a, b

In [ ]:
test_df, n = generate_test_contrct('/content/drive/MyDrive/experiments/baseline benchmark/Replica.sol', 0.95)
truth, prompt = truncate_test(test_df, n)
col = ['Target']
prompt_df = pd.DataFrame([prompt], columns=col)
gpt_out = text_generation(trained_model, tokenizer, prompt_df)
print('this is the GPT prompt without T5: \n', prompt_df['Target'][0])
print('this is the GPT prediction without T5: \n', gpt_out)